## Todo

- [x] Use `test_image.tif` and `solar_shapfile_test.shp` as input
- [x] Create mask image from both inputs
  - [x] Check the rasterized mask image
- [x] tiling on both `test_image.tif` and `rasterized_mask` for Segmentation task
- [ ] Train U-Net model with `segmentation_models` library
  - [ ] Build Docker image for training

In [13]:
import rioxarray
import geopandas as gpd
import shapely as shp
import rasterio
from geo_rasterizer import rasterize
import os
import xarray as xr

import asyncio

In [2]:
from tqdm.notebook import tqdm

In [3]:
image_path = "../data/Sample1/image/test_image.tif"
shape_path = "../data/Sample1/shapefile/solar_shapefile_test.shp"
mask_path = "../data/Sample1/mask/mask.tif"

In [2]:
def create_mask(image_path:str, shape_path:str, output_path:str):
    # Load shape
    if(os.path.exists(shape_path) == False):
        raise FileExistsError(f"{shape_path=} is not exist")
    if(os.path.exists(image_path) == False):
        raise FileExistsError(f"{image_path=} is not exist")
    
    shape = gpd.read_file(shape_path)

    # Load DatasetReader
    ds_reader = rasterio.open(image_path)
    # create raster mask image
    mask = rasterize(shape, ds_reader)
    del(ds_reader)
    del(shape)
    
    # Save mask.tif
    image = rioxarray.open_rasterio(image_path)
    ds = xr.Dataset({
        'mask':xr.DataArray(
            data=mask,
            dims = ['latitude','longitude'],
            coords = {'latitude': image.y.values ,'longitude':image.x.values}
        )
    })
    ds.rio.write_crs('epsg:32647', inplace=True)
    ds.rio.to_raster(output_path, compress='LZW')
    del(mask)
    del(ds)
    del(image)

create_mask(image_path=image_path, shape_path=shape_path, output_path=mask_path)

In [19]:
# Tiling with 512 x 512 window size

def create_tile(image_path:str, mask_path:str, tile_size:int):
    if(os.path.exists(image_path) == False):
        raise FileExistsError(f"{image_path=} is not exist")
    if(os.path.exists(mask_path) == False):
        raise FileExistsError(f"{mask_path=} is not exist")
    
    if(tile_size % 32 != 0):
        raise ValueError(f"U-Net tile size should devisible by 32 but got {tile_size=}")

    image = rioxarray.open_rasterio(image_path)
    mask = rioxarray.open_rasterio(mask_path)

    

    for index_y in tqdm(range(0,len(image.y),512)):
        for index_x in range(0,len(image.x),512):
            image_tile = image[:, index_y:index_y+tile_size , index_x:index_x+tile_size ]
            mask_tile = mask[:, index_y:index_y+tile_size , index_x:index_x+tile_size ]
            image_tile.rio.to_raster(f"../data/Sample1_tile/image/image_{index_y}_{index_x}.tif")
            mask_tile.rio.to_raster(f"../data/Sample1_tile/mask/mask_{index_y}_{index_x}.tif")
    del(image)
    del(mask)
    del(image_tile)
    del(mask_tile)

create_tile(image_path=image_path, mask_path=mask_path, tile_size=512)

  0%|          | 0/137 [00:00<?, ?it/s]